In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


def get_col_dtype(col):
        """
        Identify datatype with pandas 
        """
        if col.dtype =="object":
            try:
                col_new = pd.to_datetime(col)
                return col_new.dtype
            except:
                try:
                    col_new = pd.to_numeric(col)
                    print(col_new.dtype)
                    return col_new.dtype
                except:
                    return "Object"
        if col.dtype =="bool": return "Object"
        else:
            if col.dtype == 'float64' or col.dtype == 'int64':
                return 'Numeric'
            else:      
                return col.dtype


testdf = pd.read_csv('../../Benchmark-Labeled-Data/data_test.csv')
test_metadata = pd.read_csv('../../Benchmark-Labeled-Data/Metadata/meta_data.csv')

print(len(testdf),len(test_metadata))
test_merged = pd.merge(testdf,test_metadata,on='Record_id')
print(len(test_merged))

print(test_merged)
y_true = test_merged.y_act.values.tolist()
print(y_true)
print(len(y_true))

In [ ]:
y_pandas = []
prv_csv_name,csv_name = '',''
exception_indices = []

for index, row in test_merged.iterrows():     
    if index%100==0:  print(index)
    col = row['Attribute_name']
    prv_csv_name = csv_name
    csv_name = '../../RawCSV/' + row['name']
    
    print(csv_name)
    print(col)

    if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')
    
    try:
        curtype = get_col_dtype(df[col])
    except KeyError:
        curtype = 'Object'
    print(curtype)
    y_pandas.append(curtype)

In [ ]:
dict_label = {
    'Numeric': 0,
    "datetime64[ns]": 2,
    'datetime64[ns, UTC]': 2,
    'datetime64[ns, pytz.FixedOffset(-240)]': 2,
    'datetime64[ns, pytz.FixedOffset(-300)]': 2,
    'datetime64[ns, pytz.FixedOffset(-60)]': 2,
    'Object':8,
    'bool':8
}

dict_label_true = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

y_true = [dict_label_true[str(i)] for i in y_true]
y_pandas = [dict_label[str(i)] for i in y_pandas]
# print(y_pandas)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score


def PrintMetrics(y_true, y_pred):
    print(f'Accuracy: {accuracy_score(y_true, y_pred)}')

    matrix = confusion_matrix(y_true, y_pred)
    print(matrix)

    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 score: {f1}')


PrintMetrics(y_true, y_pandas)